In [ ]:
import torch


# Function to upload and load the model
def upload_model(model_path: str):
    state_dict = torch.load(model_path, map_location="cpu", weights_only=True)

    return state_dict


# Load the model
model_path = "/home/iony/DTU/f24/thesis/code/lgvit/LGViT-ViT-Cifar100/pytorch_model.bin"
saved_model_state_dict = upload_model(model_path)

# Display the model structure
for key, value in saved_model_state_dict.items():
    print(f"{key}: {value.shape}")

deit.embeddings.cls_token: torch.Size([1, 1, 768])
deit.embeddings.position_embeddings: torch.Size([1, 197, 768])
deit.embeddings.patch_embeddings.projection.weight: torch.Size([768, 3, 16, 16])
deit.embeddings.patch_embeddings.projection.bias: torch.Size([768])
deit.encoder.layer.0.attention.attention.query.weight: torch.Size([768, 768])
deit.encoder.layer.0.attention.attention.query.bias: torch.Size([768])
deit.encoder.layer.0.attention.attention.key.weight: torch.Size([768, 768])
deit.encoder.layer.0.attention.attention.key.bias: torch.Size([768])
deit.encoder.layer.0.attention.attention.value.weight: torch.Size([768, 768])
deit.encoder.layer.0.attention.attention.value.bias: torch.Size([768])
deit.encoder.layer.0.attention.output.dense.weight: torch.Size([768, 768])
deit.encoder.layer.0.attention.output.dense.bias: torch.Size([768])
deit.encoder.layer.0.intermediate.dense.weight: torch.Size([3072, 768])
deit.encoder.layer.0.intermediate.dense.bias: torch.Size([3072])
deit.encoder.l

In [2]:
import utils as my_utils

args = my_utils.parse_config(from_argparse=False)

# Dataset config
dataset_config = args["dataset"]
# ViT config
model_config = args["model"]

model = my_utils.get_model(model_config)

[INFO][ViT][13:12:50.662]: Initializing Vit model...
[INFO][ViT][13:12:50.663]: image_height: 256, image_width: 256
[INFO][ViT][13:12:50.663]: patch_height: 32, patch_width: 32
[INFO][ViT][13:12:50.750]: ViT model initialized


In [3]:
for key, value in model.state_dict().items():
    print(f"{key}:", "\t", value.shape)

patch_embedding.pos_embedding: 	 torch.Size([1, 65, 768])
patch_embedding.cls_token: 	 torch.Size([1, 1, 768])
patch_embedding.patch_embedding_layers.1.weight: 	 torch.Size([3072])
patch_embedding.patch_embedding_layers.1.bias: 	 torch.Size([3072])
patch_embedding.patch_embedding_layers.2.weight: 	 torch.Size([768, 3072])
patch_embedding.patch_embedding_layers.2.bias: 	 torch.Size([768])
patch_embedding.patch_embedding_layers.3.weight: 	 torch.Size([768])
patch_embedding.patch_embedding_layers.3.bias: 	 torch.Size([768])
transformer.norm.weight: 	 torch.Size([768])
transformer.norm.bias: 	 torch.Size([768])
transformer.layers.0.norm.weight: 	 torch.Size([768])
transformer.layers.0.norm.bias: 	 torch.Size([768])
transformer.layers.0.to_qkv.weight: 	 torch.Size([3072, 768])
transformer.layers.0.to_ff.0.weight: 	 torch.Size([768, 1024])
transformer.layers.0.to_ff.0.bias: 	 torch.Size([768])
transformer.layers.0.feed_forward.net.0.weight: 	 torch.Size([768])
transformer.layers.0.feed_forwa

In [15]:
lgvit_map = {
    "patch_embedding.weight": saved_model_state_dict["patch_embedding.weight"],
    "last_classifier.bias": saved_model_state_dict["classifier.bias"],
    "last_classifier.weight": saved_model_state_dict["classifier.weight"],
}
model.load_state_dict(lgvit_map)

0x7f6958581480
0x7f69581fc340


# Pytorch example

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class SubNet(nn.Module):
    def __init__(self):
        super(SubNet, self).__init__()
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.linear_layers = SubNet()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.linear_layers(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (linear_layers): SubNet(
    (fc1): Linear(in_features=400, out_features=120, bias=True)
    (fc2): Linear(in_features=120, out_features=84, bias=True)
    (fc3): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [13]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

print()

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
linear_layers.fc1.weight 	 torch.Size([120, 400])
linear_layers.fc1.bias 	 torch.Size([120])
linear_layers.fc2.weight 	 torch.Size([84, 120])
linear_layers.fc2.bias 	 torch.Size([84])
linear_layers.fc3.weight 	 torch.Size([10, 84])
linear_layers.fc3.bias 	 torch.Size([10])

Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]
